In [ ]:
!pip install deepface

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import numpy as np
import pandas as pd
import random
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inceptionv3_preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from deepface import DeepFace

In [ ]:
sns.set(rc={'axes.facecolor': '#f0f2f2'}, style='darkgrid')

In [ ]:
with_mask_path = '/kaggle/input/mask-detection/with_mask'
without_mask_path = '/kaggle/input/mask-detection/without_mask'

with_mask_images = os.listdir(with_mask_path)
without_mask_images = os.listdir(without_mask_path)

with_mask_count = len(with_mask_images)
without_mask_count = len(without_mask_images)

total_images = with_mask_count + without_mask_count
with_mask_percentage = (with_mask_count / total_images) * 100
without_mask_percentage = (without_mask_count / total_images) * 100

labels = ['With Mask', 'Without Mask']
counts = [with_mask_count, without_mask_count]
percentages = [with_mask_percentage, without_mask_percentage]

plt.figure(figsize=(15, 4))

ax = sns.barplot(y=labels, x=counts, orient='h', color='#008281')

ax.set_xticks(range(0, max(counts) + 500, 500))  

for i, p in enumerate(ax.patches):
    width = p.get_width()
    ax.text(width + 5, p.get_y() + p.get_height()/2., 
            '{:1.2f}% ({})'.format(percentages[i], counts[i]),
            va="center", fontsize=15)
    
plt.xlabel('Number of Images', fontsize=14)

plt.title("Number of images per class (Masked vs Unmasked)", fontsize=18)
plt.show()

In [ ]:
heights = []
widths = []

unique_dims = set()
unique_channels = set()

def process_images(image_path):
    filenames = os.listdir(image_path)
    for filename in filenames:
        img = cv2.imread(os.path.join(image_path, filename))
        if img is not None:
            unique_dims.add((img.shape[0], img.shape[1]))
            
            unique_channels.add(img.shape[2])
            
            heights.append(img.shape[0])
            widths.append(img.shape[1])

process_images(with_mask_path)
process_images(without_mask_path)

if len(unique_dims) == 1:
    print(f"All images have the same dimensions: {list(unique_dims)[0]}")
else:
    print(f"There are {len(unique_dims)} different image dimensions in the dataset.")
    print(f"Min height: {min(heights)}, Max height: {max(heights)}, Mean height: {np.mean(heights):.2f}")
    print(f"Min width: {min(widths)}, Max width: {max(widths)}, Mean width: {np.mean(widths):.2f}")

if len(unique_channels) == 1:
    channel = list(unique_channels)[0]
    if channel == 3:
        print("All images are color images.")
    else:
        print("All images have the same number of channels, but they are not color images.")
else:
    print("Images have different numbers of channels.")

In [ ]:
def plot_images(images, title, path):
    plt.figure(figsize=(15, 3))
    for i, img_name in enumerate(images):
        plt.subplot(1, 6, i + 1)
        img = cv2.imread(os.path.join(path, img_name))
        # Resize the image to a fixed size (e.g., 224x224)
        img = cv2.resize(img, (224, 224))
        # Convert the BGR image (default in OpenCV) to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis('off')
    plt.suptitle(title, fontsize=20)
    plt.show()

random_with_mask_images = random.sample(with_mask_images, 6)
random_without_mask_images = random.sample(without_mask_images, 6)

plot_images(random_with_mask_images, "Randomly Selected With-Mask Images", with_mask_path)
plot_images(random_without_mask_images, "Randomly Selected Without-Mask Images", without_mask_path)

In [ ]:
data = []

data.extend([(os.path.join(with_mask_path, filename), "with_mask") for filename in os.listdir(with_mask_path)])

data.extend([(os.path.join(without_mask_path, filename), "without_mask") for filename in os.listdir(without_mask_path)])

df = pd.DataFrame(data, columns=['filepath', 'label'])

df.head()

In [ ]:
print("Total number of images in the dataset:", len(df))

In [ ]:
del heights, widths, unique_dims, unique_channels, data, with_mask_images, without_mask_images

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

print("Training data shape:", train_df.shape)
print("Validation data shape:", val_df.shape)

del df

In [ ]:
train_df.head()

In [ ]:
def create_data_generators(train_df, val_df, filepath_column='filepath', label_column='label', 
                           preprocessing_function=None, batch_size=32, image_dimensions=(299, 299)):
   

   train_datagen = ImageDataGenerator(
        rotation_range=15,                             # Randomly rotate the images by up to 15 degrees
        width_shift_range=0.15,                        # Randomly shift images horizontally by up to 15% of the width
        height_shift_range=0.15,                       # Randomly shift images vertically by up to 15% of the height
        zoom_range=0.15,                               # Randomly zoom in or out by up to 15%
        horizontal_flip=True,                          # Randomly flip images horizontally
        vertical_flip=False,                           # Do not flip images vertically as it doesn't make sense in our context
        shear_range=0.02,                              # Apply slight shear transformations
        preprocessing_function=preprocessing_function  # Apply preprocessing function if provided
    )

    val_datagen = ImageDataGenerator(
        preprocessing_function=preprocessing_function
    )

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,                 # DataFrame containing training data
        x_col=filepath_column,              # Column with paths to image files
        y_col=label_column,                 # Column with image labels
        target_size=image_dimensions,       # Resize all images to size of 224x224 
        batch_size=batch_size,              # Number of images per batch
        class_mode='binary',                # Specify binary classification task
        seed=42,                            # Seed for random number generator to ensure reproducibility
        shuffle=True                        # Shuffle the data to ensure the model gets a randomized batch during training
    )

    val_generator = val_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col=filepath_column,
        y_col=label_column,
        target_size=image_dimensions,
        batch_size=batch_size,
        class_mode='binary',
        seed=42,
        shuffle=False
    )
    
    return train_generator, val_generator

In [ ]:
inceptionv3_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
x = inceptionv3_base.output
x = GlobalAveragePooling2D()(x) 
x = Dense(1024, activation='relu')(x)  
x = Dropout(0.5)(x)  
predictions = Dense(1, activation='sigmoid')(x)

inceptionv3_model = Model(inputs=inceptionv3_base.input, outputs=predictions)

inceptionv3_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
inceptionv3_model.summary()

In [ ]:
plot_model(inceptionv3_model, show_shapes=True, show_layer_names=False, dpi=100)

In [ ]:
num_epochs = 10

train_generator, val_generator = create_data_generators(train_df, 
                                                        val_df, 
                                                        preprocessing_function=inceptionv3_preprocess_input, 
                                                        batch_size=32, 
                                                        image_dimensions=(299, 299))

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3, restore_best_weights=True, verbose=1)

history = inceptionv3_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[reduce_lr, early_stopping]
)

In [ ]:
hist = pd.DataFrame(history.history)
plt.figure(figsize=(15,6))

plt.subplot(1, 2, 1)
sns.lineplot(x=hist.index+1, y=hist['loss'], color='#006766', label='Train Loss', marker='o', linestyle='--')
sns.lineplot(x=hist.index+1, y=hist['val_loss'], color='orangered', label='Validation Loss', marker='o', linestyle='--')
plt.title('Loss Evolution')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(range(0,num_epochs+1))

plt.subplot(1, 2, 2)
sns.lineplot(x=hist.index+1, y=hist['accuracy'], color='#006766', label='Train Accuracy', marker='o', linestyle='--')
sns.lineplot(x=hist.index+1, y=hist['val_accuracy'], color='orangered', label='Validation Accuracy', marker='o', linestyle='--')
plt.title('Accuracy Evolution')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.xticks(range(0,num_epochs+1))

plt.show()

In [ ]:
results = inceptionv3_model.evaluate(val_generator, steps=len(val_generator))

accuracy = results[1]

print(f'Validation Accuracy: %{round(100*accuracy,2)}')

In [ ]:
def analyze_mask_in_image(image, model):

    resized_img = cv2.resize(image, (299, 299))
    
    processed_img = inceptionv3_preprocess_input(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))

    prediction = model.predict(np.expand_dims(processed_img, axis=0))
    
    return prediction[0][0]

In [ ]:
def annotate_image_with_attributes(image, model, detector_backend='retinaface'):
    analysis = DeepFace.analyze(img_path=image, actions=['emotion', 'age', 'gender', 'race'], detector_backend=detector_backend)

    for face in analysis:
        x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        face_img = image[y:y+h, x:x+w]
        
        mask_presence = analyze_mask_in_image(face_img, model)

        if mask_presence < 0.5:  # Threshold might need adjustment
            text_line1 = "Masked"
        else:
            text_line1 = f"{face['dominant_race'].capitalize()}, {face['dominant_gender']}, Age: {face['age']}"
            text_line2 = f"Emotion: {face['dominant_emotion']}"
            cv2.putText(image, text_line2, (x, y + h + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.putText(image, text_line1, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
    return image